In [3]:
# surprise library 설치
conda install -c conda-forge scikit-surprise

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yoonji\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    scikit-surprise-1.1.1      |   py38h1e00858_1         567 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38
  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.1-py38h1e00858_1

The following packages will be UPDATED:

  conda                       pkgs



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda




In [4]:
# surprise library 업데이트
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yoonji\anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    backports.functools_lru_cache-1.6.1|     pyhd3eb1b0_0          12 KB
    conda-4.9.2                |   py38haa95532_0         2.9 MB
    conda-package-handling-1.7.2|   py38h76e460a_0         723 KB
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

  conda-package-han~                   1.6.1-py38h62dcd97_0 --> 1.7.2-py38h76e460a_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.9.2-py38haa244fe~ --> pkgs/main::conda-4.9.2-py38haa95532_0

The following packages will be DOWNGRADED:

  backports.functoo~   

In [1]:
import surprise
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 1. 데이터 로드
data = surprise.Dataset.load_builtin('ml-100k')

In [7]:
# data 어떻게 생겼는지 보려고 했는데 그냥 일반 data frame 아닌듯
data

In [4]:
 # head 같은 함수 적용할 수 없다고 나옴
data.head()

AttributeError: 'DatasetAutoFolds' object has no attribute 'head'

In [31]:
# 참고 자료를 바탕으로 data가 어떻게 생겼는지 보여주는 코드
# df를 사용하진 않음-데이터 어떻게 생겼는지 보여주기만
df = pd.DataFrame(data.raw_ratings, columns=["user", "movie", "rate", "id"])

In [32]:
del df['id']
df.head()

,user,movie,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [37]:
# 2. 모델 검증을 위한 data split
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [48]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  943 

Number of items:  1651 



In [49]:
# ?
trainset_iids = list(trainset.all_items())
iid_converter = lambda x: trainset.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [50]:
# 3. 본격 알고리즘 구현
from surprise import KNNWithMeans   

my_k = 15
my_min_k = 5

my_sim_option = {
    'name':'pearson', 'user_based':False, 
    }

algo = KNNWithMeans(
    k = my_k, min_k = my_min_k, sim_option = my_sim_option
    )

Computing the msd similarity matrix...
Done computing similarity matrix.


In [53]:
# model fit & 평가
algo.fit(trainset)

from surprise import accuracy
# testset에 적용 후 rmse 계산-작을수록 좋음
predictions = algo.test(testset)
accuracy.rmse(predictions)   

RMSE: 0.9631


0.9631483820282162

In [54]:
# cross validation을 통해 여러번 시행
from surprise.model_selection import cross_validate

results = cross_validate(
    algo = algo, data = data, measures=['RMSE'], 
    cv=5, return_train_measures=True
    )

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
# cross validation을 통해 구한 rmse들의 평균을 통해 모델 평가
results['test_rmse'].mean()

0.9630227344524316

In [56]:
# 모델이 만족스러우면 full trainset에 fit-방금한 cross validation은 일부 trainset들에 적용한것이지 full train set 아님 
trainsetfull = data.build_full_trainset()
algo.fit(trainsetfull)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [60]:
# full trainset에 fit시킨 모델을 testset에 적용
#test set=기존 데이터에서 rate 없는 부분  ex) 영화 100개 중 사용자가 15개만 평가했다면 나머지 85개가 test set
testset = trainset.build_anti_testset()   
predictions = algo.test(testset)

In [64]:
# predictions는 userid, itmid, 실제 평점, 예측 평점
predictions[:5]

[Prediction(uid='877', iid='602', r_ui=3.5318875, est=3.853421453621185, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='877', iid='431', r_ui=3.5318875, est=3.7511298591944326, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='877', iid='875', r_ui=3.5318875, est=3.409014097777476, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='877', iid='182', r_ui=3.5318875, est=4.05057053989753, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='877', iid='1074', r_ui=3.5318875, est=3.152310580548693, details={'actual_k': 15, 'was_impossible': False})]

In [68]:
# 특정 사용자의 특정 아이템에 대한 선호도 예측
algo.predict(uid='877', iid='1')     #실제 평점이 없기 때문에 None

Prediction(uid='877', iid='1', r_ui=None, est=4.26641817991319, details={'actual_k': 15, 'was_impossible': False})

In [61]:
# top N recommendation 구현
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)      #dictionary 안을 list로 채우기
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))   #사용자 id 옆에 item id와 선호도 예측값 입력

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():   #items() : dictionary의 key와 value 함께 이용
        user_ratings.sort(key=lambda x: x[1], reverse=True)  #선호도 예측값을 내림차순 정렬
        top_n[uid] = user_ratings[:n]      #상위 n번째 영화 및 선호도 예측값까지 출력

    return top_n

In [ ]:
# 참고) lambda문
a = [(1, 2), (5, 1), (0, 1), (5, 2), (3, 0)]
c = sorted(a, key = lambda x : x[0])
# c = [(0, 1), (1, 2), (3, 0), (5, 1), (5, 2)]

d = sorted(a, key = lambda x : x[1])
# d = [(3, 0), (5, 1), (0, 1), (1, 2), (5, 2)]

In [62]:
top_n = get_top_n(predictions, n=10)   #사용자별로 10개 영화 추천

In [69]:
top_n

defaultdict(list,
            {'877': [('1449', 4.996148039644726),
              ('1367', 4.858942616113902),
              ('50', 4.757175843524499),
              ('963', 4.7565635561032344),
              ('98', 4.74397926752012),
              ('1142', 4.704092319700976),
              ('169', 4.647458216045374),
              ('176', 4.618363173154341),
              ('483', 4.612818116956483),
              ('1175', 4.5940592586120825)],
             '815': [('313', 5),
              ('1449', 4.901379786013931),
              ('272', 4.837524548800074),
              ('1367', 4.7360581310755645),
              ('408', 4.6485781734749345),
              ('496', 4.644187836946238),
              ('64', 4.610180811526368),
              ('302', 4.602186169896453),
              ('320', 4.5726079086151925),
              ('169', 4.568962745186294)],
             '94': [('1449', 4.843448530548647),
              ('100', 4.671089914520026),
              ('1367', 4.61280404200529),
  

In [63]:
# 사용자별 top_10 list
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])    #top_n에서 user id와 item id들만 추출

877 ['1449', '1367', '50', '963', '98', '1142', '169', '176', '483', '1175']
815 ['313', '1449', '272', '1367', '408', '496', '64', '302', '320', '169']
94 ['1449', '100', '1367', '320', '169', '127', '182', '963', '251', '285']
416 ['1449', '1367', '272', '169', '963', '611', '408', '1169', '483', '114']
500 ['1449', '1367', '318', '480', '127', '511', '156', '1142', '896', '484']
259 ['1449', '316', '1367', '483', '187', '511', '923', '435', '127', '515']
598 ['1367', '1449', '318', '64', '83', '483', '408', '223', '320', '963']
886 ['1449', '1367', '475', '272', '1142', '511', '302', '12', '251', '603']
837 ['1449', '1367', '408', '613', '589', '50', '1062', '1142', '316', '197']
521 ['1449', '1367', '268', '285', '134', '484', '315', '1175', '61', '603']
459 ['1449', '1367', '318', '96', '12', '1512', '483', '22', '251', '520']
622 ['1449', '272', '313', '1367', '251', '306', '923', '320', '285', '318']
655 ['1449', '483', '134', '1367', '127', '98', '641', '606', '318', '484']
128